In [6]:
from pathlib import Path
from typing import List
import re

import numpy as np
import pandas as pd
import mne


# =========================
# ユーザー設定
# =========================

ROOT_DIR = Path("/Users/shunsuke/EEG_48sounds")  # ★必要なら変更

EDF_DIR = ROOT_DIR / "raw_edf"

DERIV_DIR = ROOT_DIR / "derivatives"
DERIV_DIR.mkdir(parents=True, exist_ok=True)

EPOCH_DIR_RUN = DERIV_DIR / "epochs_run"
EPOCH_DIR_RUN.mkdir(parents=True, exist_ok=True)

EPOCH_DIR_TRIAL = DERIV_DIR / "epochs_trial"
EPOCH_DIR_TRIAL.mkdir(parents=True, exist_ok=True)

EDF_ENCODING = "latin1"

EVENT_LABEL = "\x88À\x90Ã"
MARKER_TO_SOUND_SHIFT = 5.0  # イベントバーの 5 秒後に音が鳴る

TMIN = -2.0
TMAX = 12.0

EXPECTED_TRIALS_PER_RUN = 48  # チェックしたくなければ None
# =========================
# ★ 10-20（国際10–20法）の19chだけに固定
# =========================

TEN_TWENTY_19 = [
    "Fp1","Fp2",
    "F7","F3","Fz","F4","F8",
    "T7","C3","Cz","C4","T8",
    "P7","P3","Pz","P4","P8",
    "O1","O2",
]

MONTAGE_NAME = "standard_1020"   # MNEの標準10-20
STRICT_19CH = True              # 19ch揃ってないEDFはエラーにする（必要ならFalse）



# =========================
# ★ 前処理パラメータ（控えめ）
# =========================

# 0.1 Hz ハイパス：超低周波ドリフトだけ切る
HP_FREQ: float | None = 0.1
# ローパスはここではかけない（解析側で必要に応じて）
LP_FREQ: float | None = None

# 50 Hz ノッチのみ（電源ノイズ）
NOTCH_FREQS: list[float] = [50.0]

# エポック後のベースライン補正区間（音開始基準）
BASELINE: tuple[float, float] | None = (-0.2, 0.0)  # -0.2〜0 秒


# =========================
# 補助関数
# =========================

def parse_subject_and_run(edf_path: Path):
    name = edf_path.stem

    pattern = r'^(\d+)[(（](\d+)[)）](.+)音刺激$'
    m = re.match(pattern, name)

    if m:
        sub_no = m.group(1)
        ses_no = m.group(2)
        pname  = m.group(3)
        subject = f"{sub_no}_{pname}"
        run = f"run{ses_no}"
    else:
        subject = name
        run = "run1"

    return subject, run


def get_event_onsets_from_annotations(raw: mne.io.Raw, event_label: str | None):
    ann = raw.annotations
    if len(ann) == 0:
        raise RuntimeError("この EDF には annotations がありません。")

    if event_label is None:
        onsets = [a["onset"] for a in ann]
    else:
        onsets = [
            a["onset"]
            for a in ann
            if str(a["description"]) == event_label
        ]

    if len(onsets) == 0:
        raise RuntimeError(
            f"指定した EVENT_LABEL={repr(event_label)} に一致する annotation が見つかりません。"
        )

    return np.array(onsets)


def export_single_epoch_to_csv(single_epoch: mne.Epochs, csv_path: Path):
    data = single_epoch.get_data()[0]  # (n_channels, n_times)
    ch_names = single_epoch.ch_names
    times = single_epoch.times

    df = pd.DataFrame(data.T, columns=ch_names)
    df.insert(0, "time_sec", times)

    csv_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")

def build_events_from_onsets(
    marker_onsets_sec: np.ndarray,
    sfreq: float,
    raw_n_times: int,
    marker_to_sound_shift: float,
    tmin: float,
    tmax: float,
):
    """marker_onsets_sec（イベントバー）→ 音開始 events を作る。
    ついでに、エポック窓がデータ範囲からはみ出すイベントを除外する。
    """
    sound_onsets_sec = marker_onsets_sec + marker_to_sound_shift

    # ★ 切り捨てではなく四捨五入
    sound_samples = np.rint(sound_onsets_sec * sfreq).astype(int)

    # エポック範囲チェック（start >= 0 かつ end < raw_n_times）
    start_samp = sound_samples + int(np.floor(tmin * sfreq))
    end_samp   = sound_samples + int(np.ceil(tmax * sfreq))

    ok = (start_samp >= 0) & (end_samp < raw_n_times)

    n_all = len(sound_samples)
    n_ok  = int(ok.sum())
    n_ng  = n_all - n_ok

    if n_ng > 0:
        print(f"  [注意] エポック範囲外のイベントが {n_ng}/{n_all} 個あります（除外します）")

    # 時系列順の保証（念のため）
    sound_samples_ok = sound_samples[ok]
    marker_onsets_ok = marker_onsets_sec[ok]
    sound_onsets_ok  = sound_onsets_sec[ok]

    order = np.argsort(sound_samples_ok)
    if not np.all(order == np.arange(len(order))):
        print("  [注意] イベントが時系列順になっていなかったため、時系列順に並べ替えました")

    sound_samples_ok = sound_samples_ok[order]
    marker_onsets_ok = marker_onsets_ok[order]
    sound_onsets_ok  = sound_onsets_ok[order]

    EVENT_ID = 1
    events = np.column_stack([
        sound_samples_ok,
        np.zeros(len(sound_samples_ok), dtype=int),
        np.full(len(sound_samples_ok), EVENT_ID, dtype=int),
    ])

    return events, marker_onsets_ok, sound_onsets_ok

def _canon_1020_name(name: str) -> str | None:
    """
    EDF上の表記ゆれ（Fp1-A1, EEG Fp1-REF, T3 など）を
    10-20の標準名へ寄せる。対象外は None を返す。
    """
    s = str(name).strip()

    # ありがちなプレフィックスを除去（例: "EEG Fp1-REF"）
    s = re.sub(r"^(EEG|Eeg)\s+", "", s).strip()

    # まずは "-"," " で分割して先頭を採用（例: "Fp1-A1"→"Fp1"）
    base = re.split(r"[-\s]", s)[0].strip()

    # 大文字化で判定しやすく
    b = base.upper()

    # 旧名称→新名称（10-20の現行表記寄せ）
    legacy = {"T3":"T7", "T4":"T8", "T5":"P7", "T6":"P8"}
    b = legacy.get(b, b)

    # Canonical表記へ
    canonical_map = {
        "FP1":"Fp1", "FP2":"Fp2",
        "F7":"F7", "F3":"F3", "FZ":"Fz", "F4":"F4", "F8":"F8",
        "T7":"T7", "C3":"C3", "CZ":"Cz", "C4":"C4", "T8":"T8",
        "P7":"P7", "P3":"P3", "PZ":"Pz", "P4":"P4", "P8":"P8",
        "O1":"O1", "O2":"O2",
    }

    return canonical_map.get(b, None)


def pick_10_20_19ch(raw: mne.io.Raw) -> mne.io.Raw:
    """
    Rawから10-20 19chのみ抽出し、チャンネル名も10-20に統一。
    参照（A1/A2等）やPOL系処理はしない。
    """
    raw = raw.copy()

    # 目的の19chに対応する「元チャンネル名」を1つずつ拾う（重複回避）
    chosen_orig = {}
    for ch in raw.ch_names:
        canon = _canon_1020_name(ch)
        if canon in TEN_TWENTY_19 and canon not in chosen_orig:
            chosen_orig[canon] = ch

    missing = [ch for ch in TEN_TWENTY_19 if ch not in chosen_orig]
    if missing:
        msg = f"10-20の19chが揃っていません。欠損: {missing}\nraw.ch_names={raw.ch_names}"
        if STRICT_19CH:
            raise RuntimeError(msg)
        else:
            print("  [注意]", msg)

    # 存在する分だけでも順序固定でpick
    pick_order = [chosen_orig[ch] for ch in TEN_TWENTY_19 if ch in chosen_orig]
    raw.pick_channels(pick_order)

    # 10-20名へrename
    rename_dict = {chosen_orig[canon]: canon for canon in TEN_TWENTY_19 if canon in chosen_orig}
    raw.rename_channels(rename_dict)

    # すべてEEG扱いに（余計な型混在があるEDF対策）
    raw.set_channel_types({ch: "eeg" for ch in raw.ch_names}, verbose=False)

    # 10-20モンタージュ付与（位置情報）
    montage = mne.channels.make_standard_montage(MONTAGE_NAME)
    raw.set_montage(montage, on_missing="ignore", verbose=False)

    return raw

# =========================
# ★ 軽い前処理
# =========================

def preprocess_raw(raw: mne.io.Raw) -> mne.io.Raw:
    """Raw に対して壊しにくい前処理だけを行う。
       - 0.1 Hz ハイパス
       - 50 Hz ノッチ
       ※ bad チャネル補間やローパスはしない
    """
    raw = raw.copy()

    # ハイパスのみ（ローパスは LP_FREQ=None のまま）
    if HP_FREQ is not None or LP_FREQ is not None:
        raw.filter(
            l_freq=HP_FREQ,
            h_freq=LP_FREQ,
            picks="eeg",
            fir_design="firwin",
            verbose=False,
        )

    # ノッチ（電源ノイズだけ消す）
    if NOTCH_FREQS:
        if hasattr(raw, "notch_filter"):
            raw.notch_filter(
                freqs=NOTCH_FREQS,
                picks="eeg",
                verbose=False,
            )

    return raw


def preprocess_epochs(epochs: mne.Epochs) -> mne.Epochs:
    """Epochs に対してベースライン補正だけを行う。
       参照切り替えや trial drop はしない。
    """
    epochs = epochs.copy()

    if BASELINE is not None:
        epochs.apply_baseline(BASELINE)

    return epochs


# =========================
# EDF1本分の処理
# =========================

def make_epochs_from_one_file(edf_path: Path) -> pd.DataFrame:
    print(f"\n=== 処理中: {edf_path.name} ===")

    subject, run = parse_subject_and_run(edf_path)
    print(f"  subject={subject}, run={run}")

    # EDF 読み込み
    raw = mne.io.read_raw_edf(
        edf_path,
        preload=True,
        encoding=EDF_ENCODING,
        verbose=False,
    )
    # ★ 10-20の19chだけに固定（POL系は何もしない）
    raw = pick_10_20_19ch(raw)
    print(f"  10-20 19chに固定: {raw.ch_names}")


    sfreq = raw.info["sfreq"]
    print(f"  サンプリング周波数: {sfreq} Hz")

    # ★ Raw に軽い前処理
    raw = preprocess_raw(raw)
    print("  前処理: 0.1 Hz ハイパス + 50 Hz ノッチ")

    # イベント取得（イベントバー）
    marker_onsets = get_event_onsets_from_annotations(raw, EVENT_LABEL)
    print(f"  検出されたイベント数 (EVENT_LABEL='{EVENT_LABEL}'): {len(marker_onsets)}")

    if EXPECTED_TRIALS_PER_RUN is not None and len(marker_onsets) != EXPECTED_TRIALS_PER_RUN:
        print(f"  [注意] イベント数 {len(marker_onsets)} が 想定({EXPECTED_TRIALS_PER_RUN}) と異なります。")

    # ★ events作成（丸め＋範囲外除外＋時系列保証）
    events, marker_onsets_ok, sound_onsets_ok = build_events_from_onsets(
        marker_onsets_sec=marker_onsets,
        sfreq=sfreq,
        raw_n_times=raw.n_times,
        marker_to_sound_shift=MARKER_TO_SOUND_SHIFT,
        tmin=TMIN,
        tmax=TMAX,
    )
    print(f"  有効イベント数（エポック作成に使用）: {len(events)}")

    EVENT_ID = 1

    # エポック作成（baseline=None → 後で apply_baseline）
    epochs = mne.Epochs(
        raw,
        events=events,
        event_id=EVENT_ID,
        tmin=TMIN,
        tmax=TMAX,
        baseline=None,
        preload=True,
        reject_by_annotation=False,
        verbose=False,
    )

    print(f"  作成されたエポック数: {len(epochs)}")

    # ★ Epochs にベースライン補正だけ適用
    epochs = preprocess_epochs(epochs)
    print(f"  ベースライン補正: {BASELINE} 秒")

    # ===== 1) run 単位 FIF =====
    run_fname = f"{subject}_{run}-epo.fif"
    run_path = EPOCH_DIR_RUN / run_fname
    epochs.save(run_path, overwrite=True, verbose=False)
    print(f"  [RUN] エポックを保存: {run_path}")

    # ===== 2) trial 単位 FIF & CSV =====
    subject_dir = EPOCH_DIR_TRIAL / subject
    subject_fif_dir = subject_dir / "fif"
    subject_csv_dir = subject_dir / "csv"
    subject_fif_dir.mkdir(parents=True, exist_ok=True)
    subject_csv_dir.mkdir(parents=True, exist_ok=True)

    rows = []
    for trial_idx, (sample, _zero, _id) in enumerate(events, start=1):
        onset_sec = sample / sfreq
        single_epoch = epochs[trial_idx - 1: trial_idx]

        trial_base = f"{subject}_{run}_trial{trial_idx:02d}"

        trial_fif_path = subject_fif_dir / f"{trial_base}-epo.fif"
        single_epoch.save(trial_fif_path, overwrite=True, verbose=False)

        trial_csv_path = subject_csv_dir / f"{trial_base}.csv"
        export_single_epoch_to_csv(single_epoch, trial_csv_path)

        if trial_idx <= 3 or trial_idx == len(events):
            print(f"  [TRIAL] FIF : {trial_fif_path}")
            print(f"  [TRIAL] CSV : {trial_csv_path}")

        rows.append(
            {
                "subject": subject,
                "run": run,
                "trial_in_run": trial_idx,
                "event_sample_sound_onset": int(sample),
                "event_time_sound_onset_sec": onset_sec,
                "run_epoch_file": str(run_path.relative_to(ROOT_DIR)),
                "single_epoch_fif_file": str(trial_fif_path.relative_to(ROOT_DIR)),
                "single_epoch_csv_file": str(trial_csv_path.relative_to(ROOT_DIR)),
                "event_time_marker_onset_sec": float(marker_onsets_ok[trial_idx - 1]),
                "event_time_sound_onset_sec": float(sound_onsets_ok[trial_idx - 1]),
                "marker_to_sound_shift_sec": float(MARKER_TO_SOUND_SHIFT),
                "event_label_used": repr(EVENT_LABEL),

                # 監査用：前処理・切り出し設定
                "hp_freq": HP_FREQ,
                "lp_freq": LP_FREQ,
                "notch_freqs": ",".join(map(str, NOTCH_FREQS)) if NOTCH_FREQS else "",
                "baseline": str(BASELINE),
                "tmin": TMIN,
                "tmax": TMAX,

            }
        )

    df = pd.DataFrame(rows)
    return df


# =========================
# メイン
# =========================

def main():
    edf_files: List[Path] = sorted(EDF_DIR.glob("*.edf"))
    if len(edf_files) == 0:
        raise FileNotFoundError(f"{EDF_DIR} に .edf ファイルが見つかりません。")

    print("見つかった EDF ファイル数:", len(edf_files))

    all_index_dfs: List[pd.DataFrame] = []

    for edf_path in edf_files:
        df = make_epochs_from_one_file(edf_path)
        all_index_dfs.append(df)

    index_df = pd.concat(all_index_dfs, ignore_index=True)
    index_df.sort_values(["subject", "run", "trial_in_run"], inplace=True)

    index_path = DERIV_DIR / "epoch_index_by_trial.csv"
    index_df.to_csv(index_path, index=False, encoding="utf-8-sig")

    print("\n=== すべて完了しました ===")
    print("インデックス CSV:", index_path)
    print(f"総 trial 数: {len(index_df)}")


if __name__ == "__main__":
    main()


見つかった EDF ファイル数: 36

=== 処理中: 01(1)高見音刺激.edf ===
  subject=01_高見, run=run1
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(
/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(


  10-20 19chに固定: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']
  サンプリング周波数: 500.0 Hz
  前処理: 0.1 Hz ハイパス + 50 Hz ノッチ
  検出されたイベント数 (EVENT_LABEL='ÀÃ'): 48
  有効イベント数（エポック作成に使用）: 48
  作成されたエポック数: 48
Applying baseline correction (mode: mean)
  ベースライン補正: (-0.2, 0.0) 秒
  [RUN] エポックを保存: /Users/shunsuke/EEG_48sounds/derivatives/epochs_run/01_高見_run1-epo.fif
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/fif/01_高見_run1_trial01-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/csv/01_高見_run1_trial01.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/fif/01_高見_run1_trial02-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/csv/01_高見_run1_trial02.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/fif/01_高見_run1_trial03-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/deriv

/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(
/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(


  10-20 19chに固定: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']
  サンプリング周波数: 500.0 Hz
  前処理: 0.1 Hz ハイパス + 50 Hz ノッチ
  検出されたイベント数 (EVENT_LABEL='ÀÃ'): 48
  有効イベント数（エポック作成に使用）: 48
  作成されたエポック数: 48
Applying baseline correction (mode: mean)
  ベースライン補正: (-0.2, 0.0) 秒
  [RUN] エポックを保存: /Users/shunsuke/EEG_48sounds/derivatives/epochs_run/01_高見_run2-epo.fif
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/fif/01_高見_run2_trial01-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/csv/01_高見_run2_trial01.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/fif/01_高見_run2_trial02-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/csv/01_高見_run2_trial02.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/01_高見/fif/01_高見_run2_trial03-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/deriv

...

  10-20 19chに固定: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']
  サンプリング周波数: 500.0 Hz
  前処理: 0.1 Hz ハイパス + 50 Hz ノッチ
  検出されたイベント数 (EVENT_LABEL='ÀÃ'): 48
  有効イベント数（エポック作成に使用）: 48
  作成されたエポック数: 48
Applying baseline correction (mode: mean)
  ベースライン補正: (-0.2, 0.0) 秒
  [RUN] エポックを保存: /Users/shunsuke/EEG_48sounds/derivatives/epochs_run/12_濱津_run1-epo.fif
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run1_trial01-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/csv/12_濱津_run1_trial01.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run1_trial02-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/csv/12_濱津_run1_trial02.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run1_trial03-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/deriv

/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(
/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(


  10-20 19chに固定: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']
  サンプリング周波数: 500.0 Hz
  前処理: 0.1 Hz ハイパス + 50 Hz ノッチ
  検出されたイベント数 (EVENT_LABEL='ÀÃ'): 48
  有効イベント数（エポック作成に使用）: 48
  作成されたエポック数: 48
Applying baseline correction (mode: mean)
  ベースライン補正: (-0.2, 0.0) 秒
  [RUN] エポックを保存: /Users/shunsuke/EEG_48sounds/derivatives/epochs_run/12_濱津_run2-epo.fif
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run2_trial01-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/csv/12_濱津_run2_trial01.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run2_trial02-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/csv/12_濱津_run2_trial02.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run2_trial03-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/deriv

/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(
/var/folders/fv/bm2mxnh10x1206wh10wf_ft00000gn/T/ipykernel_12770/3440986143.py:304: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL DC01, POL DC02, POL DC03, POL DC04, POL Pulse, POL PI, POL PR
  raw = mne.io.read_raw_edf(


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
  10-20 19chに固定: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']
  サンプリング周波数: 500.0 Hz
  前処理: 0.1 Hz ハイパス + 50 Hz ノッチ
  検出されたイベント数 (EVENT_LABEL='ÀÃ'): 48
  有効イベント数（エポック作成に使用）: 48
  作成されたエポック数: 48
Applying baseline correction (mode: mean)
  ベースライン補正: (-0.2, 0.0) 秒
  [RUN] エポックを保存: /Users/shunsuke/EEG_48sounds/derivatives/epochs_run/12_濱津_run3-epo.fif
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run3_trial01-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/csv/12_濱津_run3_trial01.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/fif/12_濱津_run3_trial02-epo.fif
  [TRIAL] CSV : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/csv/12_濱津_run3_trial02.csv
  [TRIAL] FIF : /Users/shunsuke/EEG_48sounds/derivatives/epochs_trial/12_濱津/f